In [1]:
%matplotlib inline

import datetime as datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import calendar

plt.style.use('default') # Make the graphs a bit prettier
plt.rcParams['figure.figsize'] = (15, 5)

In [3]:
trip = pd.read_csv('data/trip_train.csv', low_memory=False)
trip['start_date'] = pd.to_datetime(trip['start_date'])
trip['start_hour'] = pd.to_datetime(trip['start_date'], format = '%H', errors = 'coerce').dt.hour
trip['dia'] = trip['start_date'].dt.strftime('%w')
trip['date'] = trip.start_date.dt.date

In [4]:
stations = pd.read_csv('./data/station.csv', low_memory=False)
stations = stations.loc[:,['id','city']]

In [5]:
tabla = pd.read_csv('./tablIdZip.csv', low_memory=True)
tabla['start_station_id'] = tabla['id']
tabla = tabla.loc[:,['start_station_id','zipCode']]

In [6]:
#tripAux2 = tripAux2[tripAux.subscription_type == 'Subscriber']

In [7]:
from sklearn import linear_model
print(__doc__)

Automatically created module for IPython interactive environment


In [8]:
salida = pd.read_csv('data/trip_test.csv', low_memory=False)
#esto tarda
salida['start_date'] = pd.to_datetime(salida['start_date'])
salida['start_hour'] = pd.to_datetime(salida['start_date'], format = '%H', errors = 'coerce').dt.hour
salida['date'] = salida['start_date'].dt.strftime('%w')

In [9]:
weather = pd.read_csv('./data/weather.csv', low_memory=False)
weather['zipCode'] = weather['zip_code']
weatherEvents = weather.loc[(weather.events > ""),("date","events",'zipCode')]
weatherEvents['date'] = pd.to_datetime(weatherEvents['date'])
weatherEvents.head()

,date,events,zipCode
14,2013-09-12,Fog,94107
23,2013-09-21,Rain,94107
32,2013-09-30,Fog,94107
44,2013-10-12,Fog,94107
52,2013-10-20,Fog,94107


In [10]:
tabla.zipCode.value_counts()

94107    35
95113    16
94063     7
94041     7
94301     5
Name: zipCode, dtype: int64

In [11]:
trip.to_csv("auxtrip.csv", index=False)
weatherEvents.to_csv("auxweather.csv", index=False)

In [12]:
trip = pd.read_csv('auxtrip.csv', low_memory=False)
weatherEvents = pd.read_csv('auxweather.csv', low_memory=False)

In [13]:
stations2 = stations
#stations2 = stations2[stations2.id != 84]
#stations2 = stations2[stations2.id != 80]
#Si no se quita ninguna son 70

In [17]:
salidaFinal = trip[trip.date == str(7)]
salidaFinal = salidaFinal.loc[:,['id','salida']]
#vorrar los viajes que vuelven al mismo lugar no sirve
#tripEntrada = trip[trip.start_station_id != trip.end_station_id]
#dividir entre cosutmer y no tampoco funciona
#Elino viajes a destinos muy "extraños"
tripEntrada = trip
#tripEntrada = auxFinal
tripsSalida = salida
#tripEntrada = tripEntrada[tripEntrada.duration > 60*2]          
tripEntrada = tripEntrada[tripEntrada.duration < 60*60*24]  

#import math
#x=float('nan')
#math.isnan(x)

T = 0;
for y in range(0, 70):
    for x in range(0, 7):
        for z in range (0,25):
            tripAuxX = tripEntrada[tripEntrada.start_station_id == stations2.id[y]]
            tripsSalidaX = tripsSalida[tripsSalida.start_station_id == stations2.id[y]]
            tripsAuxX = tripAuxX[tripAuxX.start_hour == z]
            tripsSalidaX = tripsSalidaX[tripsSalidaX.start_hour == z]            
            tripAuxX = tripAuxX[tripAuxX.dia == x]
            tripsSalidaX = tripsSalidaX[tripsSalidaX.date == str(x)]
            tripsJJ = tripAuxX[trip.date == str(7)]
            D = 30000000000000
            T=0
            for P in range (0,30):
                K = tripAuxX.count().id
                Aux = tripAuxX.end_station_id.value_counts().to_frame(name='cantidades')
                Aux['end_station_id'] = Aux.index
                tripsJJ = pd.merge(tripAuxX, Aux, on='end_station_id', how='inner')
                tripsJJ = tripsJJ[tripsJJ.cantidades >=  K/P]
                S = tripsJJ['duration'].sum()/tripsJJ.count().id  
                Aux = tripAuxX    
                Aux['salida']= S
                Aux['diferencia']=(Aux['salida']-Aux['duration'])**2
                Aux.diferencia.sum()
                if (True != math.isnan(S)):
                    if (D> Aux.diferencia.sum()):
                        D=Aux.diferencia.sum()
                        T=S
            tripsSalidaX['salida']= T
            tripsX= tripsSalidaX.loc[:,['id','salida']]
            salidaFinal = salidaFinal.append(tripsX)

/home/geco/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [18]:
salidaFinal2 = salidaFinal.sort_values(by='id', ascending=True)
salidaFinal2['duration']= salidaFinal2['salida']
salidaFinal2= salidaFinal2.loc[:,['id','duration']]
salidaFinal2.to_csv("salida.csv", index=False)

In [19]:
#119998 
salidaFinal2.count()

id          119998
duration    119998
dtype: int64

In [130]:
salidaFinal2 = pd.merge(salidaFinal2 , salida, on='id', how='left')

In [131]:
salidaFinal2 = salidaFinal2[salidaFinal2.start_station_id != 84]
salidaFinal2 = salidaFinal2[salidaFinal2.start_station_id != 80]

In [132]:
salidaFinal2= salidaFinal2.loc[:,['id','duration']]

In [133]:
tripEntrada = trip
for x in range(0, 7):
    for z in range (0,25):
        tripAuxX = tripEntrada[tripEntrada.start_station_id == 84]
        tripsSalidaX = tripsSalida[tripsSalida.start_station_id == 84]
        tripsAuxX = tripAuxX[tripAuxX.start_hour == z]
        tripsSalidaX = tripsSalidaX[tripsSalidaX.start_hour == z]            
        tripAuxX = tripAuxX[tripAuxX.dia == x]
        tripsSalidaX = tripsSalidaX[tripsSalidaX.date == str(x)]
        tripsJJ = tripAuxX[trip.date == str(7)]
        D = 30000000000000
        T=0
        for P in range (20,30):
            K = tripAuxX.count().id
            Aux = tripAuxX.end_station_id.value_counts().to_frame(name='cantidades')
            Aux['end_station_id'] = Aux.index
            tripsJJ = pd.merge(tripAuxX, Aux, on='end_station_id', how='inner')
            tripsJJ = tripsJJ[tripsJJ.cantidades >=  K/P]
            S = tripsJJ['duration'].sum()/tripsJJ.count().id  
            Aux = tripAuxX    
            Aux['salida']= S
            Aux['diferencia']=(Aux['salida']-Aux['duration'])**2
            Aux.diferencia.sum()
            if (D> Aux.diferencia.sum()):
                D=Aux.diferencia.sum()
                T=S
        T = tripsJJ['duration'].sum()/tripsJJ.count().id  
        tripsSalidaX['duration']= T
        tripsX= tripsSalidaX.loc[:,['id','duration']]
        salidaFinal2 = salidaFinal2.append(tripsX)

/home/geco/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [134]:
tripEntrada = trip
for x in range(0, 7):
    for z in range (0,25):
        tripAuxX = tripEntrada[tripEntrada.start_station_id == 80]
        tripsSalidaX = tripsSalida[tripsSalida.start_station_id == 80]
        tripsAuxX = tripAuxX[tripAuxX.start_hour == z]
        tripsSalidaX = tripsSalidaX[tripsSalidaX.start_hour == z]            
        tripAuxX = tripAuxX[tripAuxX.dia == x]
        tripsSalidaX = tripsSalidaX[tripsSalidaX.date == str(x)]
        tripsJJ = tripAuxX[trip.date == str(7)]
        D = 30000000000000
        T=0
        for P in range (20,30):
            K = tripAuxX.count().id
            Aux = tripAuxX.end_station_id.value_counts().to_frame(name='cantidades')
            Aux['end_station_id'] = Aux.index
            tripsJJ = pd.merge(tripAuxX, Aux, on='end_station_id', how='inner')
            tripsJJ = tripsJJ[tripsJJ.cantidades >=  K/P]
            S = tripsJJ['duration'].sum()/tripsJJ.count().id  
            Aux = tripAuxX    
            Aux['salida']= S
            Aux['diferencia']=(Aux['salida']-Aux['duration'])**2
            Aux.diferencia.sum()
            if (D> Aux.diferencia.sum()):
                D=Aux.diferencia.sum()
                T=S
        T = tripsJJ['duration'].sum()/tripsJJ.count().id  
        tripsSalidaX['duration']= T
        tripsX= tripsSalidaX.loc[:,['id','duration']]
        salidaFinal2 = salidaFinal2.append(tripsX)

/home/geco/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [135]:
salidaFinal2 = salidaFinal2.sort_values(by='id', ascending=True)
salidaFinal2.to_csv("salida.csv", index=False)

In [117]:
salidaFinal2 = salidaFinal2.sort_values(by='duration', ascending=True)
salidaFinal2

,id,duration
74651,573923,202.433333
26407,203092,202.433333
65188,500158,202.433333
34765,266447,202.433333
117658,896389,202.433333
65215,500322,202.433333
117719,896876,202.433333
103438,792602,202.433333
65243,500494,202.433333
65248,500507,202.433333


In [137]:
import math
x=float('nan')
math.isnan(x)

True

In [74]:
salidaFinal2[trip.start_station_id == 84]

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code,start_hour,date
0,667732,202.433333,2015-03-04 16:30:00,Santa Clara at Almaden,4,3/4/2015 16:34,San Jose Diridon Caltrain Station,2,716,Subscriber,95020,16,3
1,301553,202.433333,2014-05-28 17:35:00,Santa Clara at Almaden,4,5/28/2014 17:38,San Jose Diridon Caltrain Station,2,147,Subscriber,94041,17,3
2,376982,202.433333,2014-07-23 17:32:00,Santa Clara at Almaden,4,7/23/2014 17:36,San Jose Diridon Caltrain Station,2,708,Subscriber,94103,17,3
3,17675,202.433333,2013-09-11 11:19:00,Santa Clara at Almaden,4,9/11/2013 11:57,MLK Library,11,64,Customer,95377,11,3
4,18241,202.433333,2013-09-11 18:37:00,Santa Clara at Almaden,4,9/11/2013 18:42,San Jose Diridon Caltrain Station,2,649,Customer,NaN,18,3
5,115469,202.433333,2013-12-04 18:10:00,Santa Clara at Almaden,4,12/4/2013 18:14,San Jose Diridon Caltrain Station,2,174,Subscriber,94107,18,3
6,101994,202.433333,2013-11-20 09:58:00,Santa Clara at Almaden,4,11/20/2013 10:02,San Jose Diridon Caltrain Station,2,712,Subscriber,95126,9,3
7,648197,202.433333,2015-02-18 08:55:00,Santa Clara at Almaden,4,2/18/2015 9:03,SJSU 4th at San Carlos,12,185,Subscriber,95110,8,3
8,765918,202.433333,2015-05-13 18:30:00,Santa Clara at Almaden,4,5/13/2015 18:34,San Jose Diridon Caltrain Station,2,695,Subscriber,94103,18,3
9,838459,202.433333,2015-07-08 17:14:00,Santa Clara at Almaden,4,7/8/2015 17:18,San Jose Diridon Caltrain Station,2,680,Subscriber,94403,17,3


In [35]:
trip[trip.start_station_id == 84]

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code,start_hour,dia,date,zipCode
227,710435,478,2015-04-03 05:59:00,Ryland Park,84,4/3/2015 6:07,Santa Clara County Civic Center,80,217,Subscriber,95112,5,5,2015-04-03,95112
660,556297,435,2014-11-24 13:25:00,Ryland Park,84,11/24/2014 13:33,Santa Clara County Civic Center,80,180,Subscriber,95112,13,1,2014-11-24,95112
1274,510770,442,2014-10-22 17:45:00,Ryland Park,84,10/22/2014 17:53,San Pedro Square,6,161,Subscriber,95112,17,3,2014-10-22,95112
1374,334392,1013,2014-06-22 16:13:00,Ryland Park,84,6/22/2014 16:30,San Pedro Square,6,206,Customer,nil,16,0,2014-06-22,nil
1759,372429,279,2014-07-21 10:20:00,Ryland Park,84,7/21/2014 10:25,Japantown,9,164,Subscriber,95110,10,1,2014-07-21,95110
2026,315999,412,2014-06-09 07:59:00,Ryland Park,84,6/9/2014 8:06,San Jose Civic Center,3,143,Subscriber,5112,7,1,2014-06-09,5112
2400,559468,655,2014-11-26 13:44:00,Ryland Park,84,11/26/2014 13:55,Santa Clara County Civic Center,80,117,Subscriber,95112,13,3,2014-11-26,95112
2528,495289,516,2014-10-13 08:10:00,Ryland Park,84,10/13/2014 8:19,San Jose Civic Center,3,247,Subscriber,5112,8,1,2014-10-13,5112
2914,345098,430,2014-06-30 13:38:00,Ryland Park,84,6/30/2014 13:45,Santa Clara County Civic Center,80,699,Subscriber,95112,13,1,2014-06-30,95112
3011,879710,324,2015-08-07 07:24:00,Ryland Park,84,8/7/2015 7:30,Santa Clara County Civic Center,80,168,Subscriber,95112,7,5,2015-08-07,95112


In [20]:
tripsSalidaX = salida
tripsSalidaX[tripsSalidaX.start_station_id == stations.id[0]];

In [75]:
tripEntrada = trip[trip.duration > 60*10]          
tripEntrada = tripEntrada[tripEntrada.duration < 60*60*10]  
tripEntrada.count().id

216467

In [125]:
stations.id[58]

70

In [11]:
tripEntrada = trip
tripEntrada = tripEntrada[tripEntrada.subscription_type == 'Subscriber']  
tripEntrada

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code,start_hour,date
0,907649,396,2015-08-27 08:36:00,Harry Bridges Plaza (Ferry Building),50,8/27/2015 8:43,Embarcadero at Sansome,60,187,Subscriber,94602,8,4
1,384043,636,2014-07-28 22:06:00,Market at 10th,67,7/28/2014 22:17,Washington at Kearny,46,417,Subscriber,94133,22,1
2,316176,334,2014-06-09 08:42:00,Market at Sansome,77,6/9/2014 8:47,2nd at Folsom,62,281,Subscriber,94107,8,1
3,618874,666,2015-01-26 16:55:00,San Francisco Caltrain 2 (330 Townsend),69,1/26/2015 17:07,Temporary Transbay Terminal (Howard at Beale),55,634,Subscriber,94602,16,1
4,910977,318,2015-08-29 15:09:00,Market at 10th,67,8/29/2015 15:14,Powell Street BART,39,607,Subscriber,94709,15,6
5,522083,337,2014-10-30 07:00:00,Townsend at 7th,65,10/30/2014 7:06,San Francisco Caltrain (Townsend at 4th),70,370,Subscriber,94107,7,4
6,880809,394,2015-08-07 17:11:00,2nd at South Park,64,8/7/2015 17:17,2nd at Townsend,61,443,Subscriber,94107,17,5
7,488938,766,2014-10-08 14:10:00,Powell at Post (Union Square),71,10/8/2014 14:23,San Francisco Caltrain (Townsend at 4th),70,485,Subscriber,94107,14,3
8,899522,531,2015-08-21 07:53:00,2nd at Folsom,62,8/21/2015 8:02,San Francisco Caltrain (Townsend at 4th),70,603,Subscriber,94549,7,5
10,291053,766,2014-05-20 08:59:00,San Francisco Caltrain (Townsend at 4th),70,5/20/2014 9:12,Market at 10th,67,528,Subscriber,95112,8,2


In [104]:
salida[salida.id == 913243]

,id,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code,start_hour,date
64383,913243,2015-08-31 18:55:00,San Francisco Caltrain (Townsend at 4th),70,8/31/2015 19:07,Davis at Jackson,42,470,Subscriber,94111,18,1


In [80]:
salidaFinal.to_csv("salida.csv", index=False, encoding='utf-8')

In [53]:
tripsJJ = tripAuxX[trip.date == str(7)]
D = 30000000000000
T=0
for P in range (20,30):
    K = tripAuxX.count().id
    Aux = tripAuxX.end_station_id.value_counts().to_frame(name='cantidades')
    Aux['end_station_id'] = Aux.index
    tripsJJ = pd.merge(tripAuxX, Aux, on='end_station_id', how='inner')
    tripsJJ = tripsJJ[tripsJJ.cantidades >=  K/P]
    S = tripsJJ['duration'].sum()/tripsJJ.count().id  
    Aux = tripAuxX    
    Aux['salida']= S
    Aux['diferencia']=(Aux['salida']-Aux['duration'])**2
    Aux.diferencia.sum()
    if (D> Aux.diferencia.sum()):
        D=Aux.diferencia.sum()
        T=S
    
T

/home/geco/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


26

In [33]:
Aux = tripAuxX.end_station_id.value_counts().to_frame(name='cantidad')
K = tripAuxX.count().id
Aux['end_station_id'] = Aux.index+1
Aux

,cantidad,end_station_id
6,31,7
84,25,85
9,18,10
2,12,3
7,9,8
3,8,4
11,5,12
10,5,11
80,3,81
14,3,15


In [34]:
Promedios = tripAuxX['end_station_id'].value_counts()/tripAuxX['end_station_id'].count()
Promedios

6     0.242188
84    0.195312
9     0.140625
2     0.093750
7     0.070312
3     0.062500
11    0.039062
10    0.039062
80    0.023438
14    0.023438
8     0.023438
4     0.023438
5     0.015625
13    0.007812
Name: end_station_id, dtype: float64

In [15]:
tripAuxX

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code,start_hour,date
3042,881355,269,2015-08-08 11:25:00,Ryland Park,84,8/8/2015 11:30,San Pedro Square,6,168,Subscriber,95112,11,6
37609,682110,379,2015-03-14 18:16:00,Ryland Park,84,3/14/2015 18:23,San Pedro Square,6,126,Subscriber,95112,18,6
40174,554611,957,2014-11-22 12:39:00,Ryland Park,84,11/22/2014 12:55,San Pedro Square,6,19,Customer,95747,12,6
45354,536235,251,2014-11-08 20:30:00,Ryland Park,84,11/8/2014 20:34,Japantown,9,205,Subscriber,95110,20,6
50488,411293,784,2014-08-16 08:52:00,Ryland Park,84,8/16/2014 9:05,Ryland Park,84,716,Subscriber,95112,8,6
50851,381027,670,2014-07-26 08:54:00,Ryland Park,84,7/26/2014 9:05,Arena Green / SAP Center,14,144,Customer,95112,8,6
51618,881710,335,2015-08-08 19:35:00,Ryland Park,84,8/8/2015 19:41,San Pedro Square,6,206,Subscriber,95112,19,6
51652,472256,768,2014-09-27 11:33:00,Ryland Park,84,9/27/2014 11:46,San Jose Diridon Caltrain Station,2,195,Subscriber,95112,11,6
58718,270449,1572,2014-05-03 11:56:00,Ryland Park,84,5/3/2014 12:22,San Pedro Square,6,143,Customer,91360,11,6
60483,472155,3985,2014-09-27 09:03:00,Ryland Park,84,9/27/2014 10:09,Ryland Park,84,301,Customer,95110,9,6


In [25]:
#Agregar Horas.
salidaFinal = trip[trip.date == str(7)]
salidaFinal = salidaFinal.loc[:,['id','salida']]
#prueba sin horas
tripEntrada = trip
T = 0;
for y in range(0, 70):
    for x in range(0, 7):
        tripAuxX = tripEntrada[tripEntrada.start_station_id == stations.id[y]]
        tripsSalidaX = salida[salida.start_station_id == stations.id[y]]         
        tripAuxX = tripAuxX[tripAuxX.date == str(x)]
        tripsSalidaX = tripsSalidaX[tripsSalidaX.date == str(x)]
        T = tripAuxX['duration'].sum()/tripAuxX.count().id
        tripsSalidaX['salida']= T
        tripsX= tripsSalidaX.loc[:,['id','salida']]
        salidaFinal = salidaFinal.append(tripsX)

In [93]:
#copias de rezerva

linea = tripAux3.loc[:,['duration','distancia']]
lineaX = linea.loc[:,['duration']]
lineaY = linea.loc[:,['distancia']]

# Se separa las variables explicadas
linea_X_train = lineaX[:-30]
linea_X_test = lineaX[-30:]
linea_Y_train = lineaY[:-30]
linea_Y_test = lineaY[-30:]

regr = linear_model.LinearRegression()
regr.fit(linea_X_train, linea_Y_train)

print (u'Regresión Mínimos Cuadrados Ordinarios')
# Coeficiente
print('Coeficientes:',regr.coef_)
# MSE
print("Residual sum of squares: %.2f" % np.mean((regr.predict(linea_X_test) - linea_Y_test) ** 2))
# Varianza explicada
print('Varianza explicada: %.2f\n' % regr.score(linea_X_test, linea_Y_test))






#Agregar Horas.
salidaFinal = trip[trip.date == str(7)]
salidaFinal = salidaFinal.loc[:,['id','salida']]
T = 0;
for y in range(0, 64):
    for x in range(0, 6):
        for z in range (0,24):
            tripAuxX = trip[trip.start_station_id == stations.id[y]]
            tripsSalidaX = salida[salida.start_station_id == stations.id[y]]
            tripsAuxX = tripAuxX[tripAuxX.start_hour == z]
            tripsSalidaX = tripsSalidaX[tripsSalidaX.start_hour == z]            
            tripAuxX = tripAuxX[tripAuxX.date == str(x)]
            tripsSalidaX = tripsSalidaX[tripsSalidaX.date == str(x)]
            T = tripAuxX['duration'].sum()/tripAuxX.count().id
            tripsSalidaX['salida']= T
            tripsX= tripsSalidaX.loc[:,['id','salida']]
            salidaFinal = salidaFinal.append(tripsX)

Regresión Mínimos Cuadrados Ordinarios
Coeficientes: [[ 0.02817081]]
Residual sum of squares: 13533.17
Varianza explicada: 0.08



In [67]:
salidaFinal = trip[trip.date == str(7)]
salidaFinal = salidaFinal.loc[:,['id','salida']]
#vorrar los viajes que vuelven al mismo lugar no sirve
#tripEntrada = trip[trip.start_station_id != trip.end_station_id]
#dividir entre cosutmer y no tampoco funciona
tripEntrada = trip
T = 0;
for y in range(0, 70):
    for x in range(0, 7):
        for z in range (0,25):
            tripAuxX = tripEntrada[tripEntrada.start_station_id == stations.id[y]]
            tripsSalidaX = salida[salida.start_station_id == stations.id[y]]
            tripsAuxX = tripAuxX[tripAuxX.start_hour == z]
            tripsSalidaX = tripsSalidaX[tripsSalidaX.start_hour == z]            
            tripAuxX = tripAuxX[tripAuxX.date == str(x)]
            tripsSalidaX = tripsSalidaX[tripsSalidaX.date == str(x)]
            T = tripAuxX['duration'].sum()/tripAuxX.count().id
            tripsSalidaX['salida']= T
            tripsX= tripsSalidaX.loc[:,['id','salida']]
            salidaFinal = salidaFinal.append(tripsX)

In [10]:
salidaFinal = trip[trip.date == str(7)]
salidaFinal = salidaFinal.loc[:,['id','salida']]
#vorrar los viajes que vuelven al mismo lugar no sirve
#tripEntrada = trip[trip.start_station_id != trip.end_station_id]
#dividir entre cosutmer y no tampoco funciona
#Elino viajes a destinos muy "extraños"
tripEntrada = trip

T = 0;
for y in range(0, 70):
    for x in range(0, 7):
        for z in range (0,25):
            tripAuxX = tripEntrada[tripEntrada.start_station_id == stations.id[y]]
            tripsSalidaX = salida[salida.start_station_id == stations.id[y]]
            tripsAuxX = tripAuxX[tripAuxX.start_hour == z]
            tripsSalidaX = tripsSalidaX[tripsSalidaX.start_hour == z]            
            tripAuxX = tripAuxX[tripAuxX.date == str(x)]
            tripsSalidaX = tripsSalidaX[tripsSalidaX.date == str(x)]
            tripsJJ = tripAuxX[trip.date == str(7)]

            K = tripAuxX.count().id
            Aux = tripAuxX.end_station_id.value_counts().to_frame(name='cantidades')
            Aux['end_station_id'] = Aux.index
            tripsJJ = pd.merge(tripAuxX, Aux, on='end_station_id', how='inner')
            tripsJJ = tripsJJ[tripsJJ.cantidades >= K/30]
            T = tripsJJ['duration'].sum()/tripsJJ.count().id  
            tripsSalidaX['salida']= T
            tripsX= tripsSalidaX.loc[:,['id','salida']]
            salidaFinal = salidaFinal.append(tripsX)

/home/geco/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [ ]:
salidaFinal = trip[trip.date == str(7)]
salidaFinal = salidaFinal.loc[:,['id','salida']]
#vorrar los viajes que vuelven al mismo lugar no sirve
#tripEntrada = trip[trip.start_station_id != trip.end_station_id]
#dividir entre cosutmer y no tampoco funciona
#Elino viajes a destinos muy "extraños"
tripEntrada = trip

T = 0;
for y in range(0, 70):
    for x in range(0, 7):
        for z in range (0,25):
            tripAuxX = tripEntrada[tripEntrada.start_station_id == stations.id[y]]
            tripsSalidaX = salida[salida.start_station_id == stations.id[y]]
            tripsAuxX = tripAuxX[tripAuxX.start_hour == z]
            tripsSalidaX = tripsSalidaX[tripsSalidaX.start_hour == z]            
            tripAuxX = tripAuxX[tripAuxX.date == str(x)]
            tripsSalidaX = tripsSalidaX[tripsSalidaX.date == str(x)]
            tripsJJ = tripAuxX[trip.date == str(7)]
            D = 30000000000000
            T=0
            for P in range (20,30):
                K = tripAuxX.count().id
                Aux = tripAuxX.end_station_id.value_counts().to_frame(name='cantidades')
                Aux['end_station_id'] = Aux.index
                tripsJJ = pd.merge(tripAuxX, Aux, on='end_station_id', how='inner')
                tripsJJ = tripsJJ[tripsJJ.cantidades >=  K/P]
                S = tripsJJ['duration'].sum()/tripsJJ.count().id  
                Aux = tripAuxX    
                Aux['salida']= S
                Aux['diferencia']=(Aux['salida']-Aux['duration'])**2
                Aux.diferencia.sum()
                if (D> Aux.diferencia.sum()):
                    D=Aux.diferencia.sum()
                    T=S
            T = tripsJJ['duration'].sum()/tripsJJ.count().id  
            tripsSalidaX['salida']= T
            tripsX= tripsSalidaX.loc[:,['id','salida']]
            salidaFinal = salidaFinal.append(tripsX)

In [ ]:
salidaFinal = trip[trip.date == str(7)]
salidaFinal = salidaFinal.loc[:,['id','salida']]
#vorrar los viajes que vuelven al mismo lugar no sirve
#tripEntrada = trip[trip.start_station_id != trip.end_station_id]
#dividir entre cosutmer y no tampoco funciona
#Elino viajes a destinos muy "extraños"
tripEntrada = trip
#tripEntrada = tripEntrada[tripEntrada.duration > 60*2]          
tripEntrada = tripEntrada[tripEntrada.duration < 60*60*24]  

T = 0;
for y in range(0, 70):
    for x in range(0, 7):
        for z in range (0,25):
            tripAuxX = tripEntrada[tripEntrada.start_station_id == stations.id[y]]
            tripsSalidaX = salida[salida.start_station_id == stations.id[y]]
            tripsAuxX = tripAuxX[tripAuxX.start_hour == z]
            tripsSalidaX = tripsSalidaX[tripsSalidaX.start_hour == z]            
            tripAuxX = tripAuxX[tripAuxX.date == str(x)]
            tripsSalidaX = tripsSalidaX[tripsSalidaX.date == str(x)]
            tripsJJ = tripAuxX[trip.date == str(7)]
            D = 30000000000000
            T=0
            for P in range (20,30):
                K = tripAuxX.count().id
                Aux = tripAuxX.end_station_id.value_counts().to_frame(name='cantidades')
                Aux['end_station_id'] = Aux.index
                tripsJJ = pd.merge(tripAuxX, Aux, on='end_station_id', how='inner')
                tripsJJ = tripsJJ[tripsJJ.cantidades >=  K/P]
                S = tripsJJ['duration'].sum()/tripsJJ.count().id  
                Aux = tripAuxX    
                Aux['salida']= S
                Aux['diferencia']=(Aux['salida']-Aux['duration'])**2
                Aux.diferencia.sum()
                if (D> Aux.diferencia.sum()):
                    D=Aux.diferencia.sum()
                    T=S
            T = tripsJJ['duration'].sum()/tripsJJ.count().id  
            tripsSalidaX['salida']= T
            tripsX= tripsSalidaX.loc[:,['id','salida']]
            salidaFinal = salidaFinal.append(tripsX)

In [26]:
salidaFinal = trip[trip.date == str(7)]
salidaFinal = salidaFinal.loc[:,['id','salida']]
#vorrar los viajes que vuelven al mismo lugar no sirve
#tripEntrada = trip[trip.start_station_id != trip.end_station_id]
#dividir entre cosutmer y no tampoco funciona
#Elino viajes a destinos muy "extraños"
tripEntrada = trip
tripsSalidaX = salida
tripEntrada = tripEntrada[tripEntrada.subscription_type == 'Subscriber']  
tripsSalidaX  = tripsSalidaX [tripsSalidaX.subscription_type == 'Subscriber']  
#tripEntrada = tripEntrada[tripEntrada.duration > 60*2]          
tripEntrada = tripEntrada[tripEntrada.duration < 60*60*24]  

T = 0;
for y in range(0, 70):
    for x in range(0, 7):
        for z in range (0,25):
            tripAuxX = tripEntrada[tripEntrada.start_station_id == stations.id[y]]
            tripsSalidaX = tripsSalidaX[tripsSalidaX.start_station_id == stations.id[y]]
            tripsAuxX = tripAuxX[tripAuxX.start_hour == z]
            tripsSalidaX = tripsSalidaX[tripsSalidaX.start_hour == z]            
            tripAuxX = tripAuxX[tripAuxX.date == str(x)]
            tripsSalidaX = tripsSalidaX[tripsSalidaX.date == str(x)]
            tripsJJ = tripAuxX[trip.date == str(7)]
            D = 30000000000000
            T=0
            for P in range (20,30):
                K = tripAuxX.count().id
                Aux = tripAuxX.end_station_id.value_counts().to_frame(name='cantidades')
                Aux['end_station_id'] = Aux.index
                tripsJJ = pd.merge(tripAuxX, Aux, on='end_station_id', how='inner')
                tripsJJ = tripsJJ[tripsJJ.cantidades >=  K/P]
                S = tripsJJ['duration'].sum()/tripsJJ.count().id  
                Aux = tripAuxX    
                Aux['salida']= S
                Aux['diferencia']=(Aux['salida']-Aux['duration'])**2
                Aux.diferencia.sum()
                if (D> Aux.diferencia.sum()):
                    D=Aux.diferencia.sum()
                    T=S
            T = tripsJJ['duration'].sum()/tripsJJ.count().id  
            tripsSalidaX['salida']= T
            tripsX= tripsSalidaX.loc[:,['id','salida']]
            salidaFinal = salidaFinal.append(tripsX)

/home/geco/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [27]:
tripEntrada = trip
tripsSalidaX = salida
tripEntrada = tripEntrada[tripEntrada.subscription_type != 'Subscriber']  
tripsSalidaX  = tripsSalidaX [tripsSalidaX.subscription_type != 'Subscriber']  
T = 0;
for y in range(0, 70):
    for x in range(0, 7):
        for z in range (0,25):
            tripAuxX = tripEntrada[tripEntrada.start_station_id == stations.id[y]]
            tripsSalidaX = salida[salida.start_station_id == stations.id[y]]
            tripsAuxX = tripAuxX[tripAuxX.start_hour == z]
            tripsSalidaX = tripsSalidaX[tripsSalidaX.start_hour == z]            
            tripAuxX = tripAuxX[tripAuxX.date == str(x)]
            tripsSalidaX = tripsSalidaX[tripsSalidaX.date == str(x)]
            T = tripAuxX['duration'].sum()/tripAuxX.count().id
            tripsSalidaX['salida']= T
            tripsX= tripsSalidaX.loc[:,['id','salida']]
            salidaFinal = salidaFinal.append(tripsX)

In [126]:
tripEntrada = trip
tripEntrada['zipCode'] = tripEntrada['zip_code']
#tripEntrada = pd.merge(tripEntrada, weatherEvents, on='start_station_id', how='outer')
aux1 = tripEntrada[tripEntrada.zipCode == '94107']
aux2 = weatherEvents[weatherEvents.zipCode == 94107]
auxFinal =  pd.merge(aux1, aux2, on='date', how='left')
aux1 = tripEntrada[tripEntrada.zipCode == '95113']
aux2 = weatherEvents[weatherEvents.zipCode == 95113]
aux =  pd.merge(aux1, aux2, on='date', how='left')
auxFinal = auxFinal.append(aux)
aux1 = tripEntrada[tripEntrada.zipCode == '94063']
aux2 = weatherEvents[weatherEvents.zipCode == 94063]
aux =  pd.merge(aux1, aux2, on='date', how='left')
auxFinal = auxFinal.append(aux)
aux1 = tripEntrada[tripEntrada.zipCode == '94041']
aux2 = weatherEvents[weatherEvents.zipCode == 94041]
aux =  pd.merge(aux1, aux2, on='date', how='left')
auxFinal = auxFinal.append(aux)
aux1 = tripEntrada[tripEntrada.zipCode == '94301']
aux2 = weatherEvents[weatherEvents.zipCode == 94301]
aux =  pd.merge(aux1, aux2, on='date', how='left')
auxFinal = auxFinal.append(aux)

auxFinal = auxFinal[auxFinal.events != 'Frog']
auxFinal

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code,start_hour,dia,date,zipCode_x,events,zipCode_y
0,316176,334,2014-06-09 08:42:00,Market at Sansome,77,6/9/2014 8:47,2nd at Folsom,62,281,Subscriber,94107,8,1,2014-06-09,94107,NaN,NaN
1,522083,337,2014-10-30 07:00:00,Townsend at 7th,65,10/30/2014 7:06,San Francisco Caltrain (Townsend at 4th),70,370,Subscriber,94107,7,4,2014-10-30,94107,NaN,NaN
2,880809,394,2015-08-07 17:11:00,2nd at South Park,64,8/7/2015 17:17,2nd at Townsend,61,443,Subscriber,94107,17,5,2015-08-07,94107,NaN,NaN
3,488938,766,2014-10-08 14:10:00,Powell at Post (Union Square),71,10/8/2014 14:23,San Francisco Caltrain (Townsend at 4th),70,485,Subscriber,94107,14,3,2014-10-08,94107,NaN,NaN
4,398366,525,2014-08-07 10:09:00,San Francisco Caltrain 2 (330 Townsend),69,8/7/2014 10:17,Embarcadero at Folsom,51,592,Subscriber,94107,10,4,2014-08-07,94107,NaN,NaN
5,863630,301,2015-07-27 15:20:00,2nd at Townsend,61,7/27/2015 15:25,San Francisco Caltrain (Townsend at 4th),70,524,Subscriber,94107,15,1,2015-07-27,94107,NaN,NaN
6,390681,528,2014-08-01 16:57:00,Beale at Market,56,8/1/2014 17:06,2nd at Townsend,61,507,Subscriber,94107,16,5,2014-08-01,94107,NaN,NaN
7,523608,192,2014-10-30 18:31:00,Spear at Folsom,49,10/30/2014 18:34,Embarcadero at Bryant,54,320,Subscriber,94107,18,4,2014-10-30,94107,NaN,NaN
8,548982,488,2014-11-18 09:11:00,Harry Bridges Plaza (Ferry Building),50,11/18/2014 9:19,2nd at Townsend,61,391,Subscriber,94107,9,2,2014-11-18,94107,NaN,NaN
9,270253,231,2014-05-02 21:26:00,San Francisco Caltrain 2 (330 Townsend),69,5/2/2014 21:30,Townsend at 7th,65,611,Subscriber,94107,21,5,2014-05-02,94107,Fog,94107.0
